Konlpy 라이브러리를 만나게된 것은 인턴 당시 텍스트 데이터를 접합으로써, 그 안에서의 키워드를 도출하여 네트워크 맵을 그리거나, 그 안에서의 빈출 단어를 추출하는 등 기본적인 텍스트 마이닝 작업을 하는데 많이 사용하였습니다. 제가 직접 언어적 특성을 기반으로 하는 함수를 생성하지 않고도, 누구나 손쉽게 문장 분리 및 형태소 분석을 진행할 수 있다는 점이 굉장히 매력적이었습니다.

Konlpy 손쉬운 사용 및 간편함에 있다고 생각합니다. 하지만, 이런 간편함은 초기에는 작은 용량의 데이터를 다룰 때는 그 장점을 누릴 수 있었지만, 대용량 데이터를 다루면서는 생각이 바뀌었습니다. 비록 코드를 짜거나 사용에는 큰 문제가 없었지만, 데이터를 돌리는데 필요한 시간이 기하급수적으로 증가하였습니다. 다행히 Konlpy내에는 Mecab 형태소 분석 엔진을 제공하여, 비교적 빠른 시간내에 처리할 수 있었습니다. 다만, 형태소 분석의 정확성에 있어 아쉬움이 항상 남아있었던 것 같습니다.

추가적으로,현재 한국어 어휘는 이전과 달리 기하급수적으로 신종어, 새로운 단어들이 1020세대를 중심으로 나타나고 있으며, 이러한 단어들을 일일히 품사 분석을 통한 분석에는 한계가 있음을 할 수 있어 아쉬웠습니다. 분야마다 필요한 Userdictionary 구성을 통해 진행할 수도 있겠습니다만, 여전히 이런 방식으로 한계성이나 확장성 측면에서 아쉬움이 있을 수 밖에 없다고 생각합니다. 

이번 오픈소스 컨트리뷰션 프로젝트 소개 자료 및 사전학습을 통해 Khaiii, kiwi 등 다른 다양한 라이브러리가 있음을 알게되었습니다. 비록 아직 직접 사용해본 경험은 부족합니다만, 이번 기회를 통해 다양한 라이브러리를 학습하고, 그 과정을 통해 제가 할 수 있는 작고 의미있는 기여들을 찾아내어 좀 더 나은 한국어 자연어처리 환경을 구축해보고 싶습니다. 

#### 라이브러리 로드

In [56]:
#기본 전처리 라이브러리
import pandas as pd

#konlpy 자연어처리 라이브러리 로드
from konlpy.tag import Kkma #태깅
from konlpy.tag import Komoran #태깅
from konlpy.tag import Hannanum #태깅
from konlpy.tag import Okt #태깅
from konlpy.utils import pprint #출력

import re #정규식 연산
import time #시간 측정

In [57]:
kkma = Kkma() 
komoran = Komoran() 
hannanum = Hannanum() 
okt = Okt() 

#### 데이터 로드 (크롤링을 통한 먹거리관련 기사 크롤링)

In [2]:
data = pd.read_csv('먹거리_식품_안전(2017~2012).csv', encoding='cp949')

In [31]:
data2 = data.dropna()
data3 = data2[:100]

#### 형태소 분석기를 활용한 체언 추출

####  N 묶음
- [NN] NNG 일반명사
- [NN] NNP 고유명사
- [NN] NNB 의존명사
- [NR] NR 수사
- [NP] NP 대명사

#### V묶음
- [VV] VV 동사
- {VA] VA 형용사
- [VX] VXV 보조 용언
- [VX] VXA 보조 형용사
- [VC] VCP 긍정 지정사, 서술격 조사 '이다'
- [VC] VCN 부정 지정사, 형용사 '아니다'

#### M묶음
- [MD] MDT 일반 관형사
- [MD] MDN 수 관형사
- [MA] MAG 일반 부사
- [MA] MAC 접속 부사

#### I묶음
- [IC] IC 감탄사

#### J묶음
- [JK] JKS 주격 조사
- [JK] JKC 보격 조사
- [JK] JKG 관형격 조사
- [JK] JKO 부사격 조사
- [JK] JKB 부사격 조사
- [JK] JKV 호격 조사
- [JK] JKQ 인용격 조사
- [JX] JX 보조사
- [JC] JC 접속 조사



- 1) Hannanum 형태소 분석기

In [58]:
%%time
full_word_list = []
full_sentence_list = []
for j in data3['Column4']:
    word_list = []
    sentence = ''
    text_analyze = hannanum.pos(j)
    
    for i in text_analyze:
        if 'N' in i:
            if i[0].isalpha() == True: #문자로만 구성되어 있는지 확인
                hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') #기호와 영어 모두 삭제 한글만 남기기
                word = hangul.sub('', i[0])
                #word = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', i[0])
                if word != '' and word != ' ' and len(word) != 1:
                    word_list.append(word)
                    sentence = sentence + ' '+ word
        
    full_word_list.append(word_list)
    full_sentence_list.append(sentence)

Wall time: 6.86 s


- 2) Komoran 형태소 분석기

In [59]:
%%time
full_word_list = []
full_sentence_list = []
for j in data3['Column4']:
    word_list = []
    sentence = ''
    text_analyze = komoran.pos(j)
    
    for i in text_analyze:
        if 'N' in i:
            if i[0].isalpha() == True: #문자로만 구성되어 있는지 확인
                hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') #기호와 영어 모두 삭제 한글만 남기기
                word = hangul.sub('', i[0])
                #word = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', i[0])
                if word != '' and word != ' ' and len(word) != 1:
                    word_list.append(word)
                    sentence = sentence + ' '+ word
        
    full_word_list.append(word_list)
    full_sentence_list.append(sentence)

Wall time: 2.1 s


- 3) Kkma 형태소 분석기

In [61]:
%%time
full_word_list = []
full_sentence_list = []
for j in data3['Column4']:
    word_list = []
    sentence = ''
    text_analyze = kkma.pos(j)
    
    for i in text_analyze:
        if 'N' in i:
            if i[0].isalpha() == True: #문자로만 구성되어 있는지 확인
                hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') #기호와 영어 모두 삭제 한글만 남기기
                word = hangul.sub('', i[0])
                #word = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', i[0])
                if word != '' and word != ' ' and len(word) != 1:
                    word_list.append(word)
                    sentence = sentence + ' '+ word
        
    full_word_list.append(word_list)
    full_sentence_list.append(sentence)

Wall time: 57.1 s


- 4) Okt 형태소 분석기

In [67]:
%%time
full_word_list = []
full_sentence_list = []
for j in data3['Column4']:
    word_list = []
    sentence = ''
    text_analyze = okt.pos(j)
    
    for i in text_analyze:
        if 'Noun' in i:
            if i[0].isalpha() == True: #문자로만 구성되어 있는지 확인
                hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') #기호와 영어 모두 삭제 한글만 남기기
                word = hangul.sub('', i[0])
                #word = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', i[0])
                if word != '' and word != ' ' and len(word) != 1:
                    word_list.append(word)
                    sentence = sentence + ' '+ word
        
    full_word_list.append(word_list)
    full_sentence_list.append(sentence)

Wall time: 3.47 s


In [69]:
# full_word_list

#### 특수한 케이스 분석
- [ 시나리오: 서술형 문장이 아닌 단어위주로 토막나 있는 경우]
- '▲박양순씨 별세ㆍ이해찬(더불어민주당 의원)씨 모친상=31일 오전9시 세종 은하수장례식장 발인 2일 오전8시 (044)901-1600▲정금선씨 별세ㆍ박남일(전 대한항공 홍보실장) 남명(아세아제지 상무)씨 모친상=31일 오전8시46분 일산병원 발인 2일 오전7시30분 (031)900-0444▲유호초씨 별세ㆍ김종천(전 대전일보 편집국장) 종휘(한국에너지기술연구원 책임연구원)씨 모친상ㆍ오병남(전 오내과의원 원장)씨 장모상=31일 오전2시30분 대전 나진요양병원 발인 3일 오전9시 (042)520-6690▲김분선씨 별세ㆍ최원영(경기문화재단 실학박물관) 원석(샌프란시스코총영사관 영사) 병선(경북 경주우체국) 병희(식품의약품안전처 안전평가원) 병숙(경주시체육회 이사) 기수씨 모친상=31일 오후1시25시 신촌세브란스병원 발인 3일 오전7시 (02)2227-7597▲이방강씨 별세ㆍ김현수(전 중소기업진흥공단 본부장)씨 부인상ㆍ주성(현대자동차 부품대리점)씨 모친상ㆍ이승국(서울화이트치과 원장)씨 장모상=31일 오전11시40분 삼성서울병원 발인 2일 오전7시40분 (02)3410-6901'

- 홍보실장을 하나의 단어로 잡아낼 수 있는가?
- 장례식장을 하나의 단어로 잡아낼 수 있는가?
- 대한항공을 하나의 단어로 잡아낼 수 있는가?
- '박양순' 이름을 하나의 단어로 분리하지 않고 잡아낼 수 있는가?

Hannanum 
- 은하수장례식장으로 장례식장으로 분리하지 못함
- 홍보실장의 경우 하나의 명사 단어로 잡아냄
- 대한항공의 경우 역시 하나의 명사 단어로 잡아냄 
- '박양순씨'로 이름과 존칭과 호칭을 하나의 명사 단어로 잡아냄

In [95]:
sent1 = data3['Column4'][0][:150]
print(hannanum.pos(sent1))
#명사 추출 외에는 큰 성과를 거두지 못했음

[('▲박양순씨', 'N'), ('별세ㆍ이해찬(더불어민주당', 'N'), ('의원', 'N'), (')', 'S'), ('씨', 'N'), ('모친', 'N'), ('상', 'X'), ('=', 'S'), ('31일', 'N'), ('오전9시', 'N'), ('세종', 'N'), ('은하수장례식장', 'N'), ('발', 'N'), ('이', 'J'), ('ㄴ', 'E'), ('2일', 'N'), ('오전8시', 'N'), ('(044)901-1600▲정금선씨', 'N'), ('별세ㆍ박남일(전', 'N'), ('대한항공', 'N'), ('홍보실장', 'N'), (')', 'S'), ('남명(아세아제지', 'N'), ('상무', 'N'), (')', 'S'), ('씨', 'N'), ('모친', 'N'), ('상', 'X'), ('=', 'S'), ('31일', 'N'), ('오전8시46분', 'N'), ('일산병원', 'N'), ('발', 'N'), ('이', 'J'), ('ㄴ', 'E'), ('2일', 'N'), ('오전7시30분', 'N'), ('(', 'S'), ('0', 'N')]


Okt
- 장례식장을 장례식과 장으로 분리해버림
- 홍보실장의 경우 홍보와 실장으로 분리해버림
- 대한항공의 경우 하나의 명사단어로 잡아냄
- 박양순 이름의 경우 박양과 순으로 분리해버림

In [75]:
print(okt.pos(sent1))
#전체적으로, 명사, 기호, 숫자 등 크게는 분리할 수 있었음

[('▲', 'Foreign'), ('박양', 'Noun'), ('순', 'Modifier'), ('씨', 'Noun'), ('별세', 'Noun'), ('ㆍ', 'Foreign'), ('이해찬', 'Noun'), ('(', 'Punctuation'), ('더불어', 'Verb'), ('민주당', 'Noun'), ('의원', 'Noun'), (')', 'Punctuation'), ('씨', 'Noun'), ('모친상', 'Noun'), ('=', 'Punctuation'), ('31일', 'Number'), ('오전', 'Noun'), ('9시', 'Number'), ('세종', 'Noun'), ('은하수', 'Noun'), ('장례식', 'Noun'), ('장', 'Suffix'), ('발인', 'Noun'), ('2일', 'Number'), ('오전', 'Noun'), ('8시', 'Number'), ('(', 'Punctuation'), ('044', 'Number'), (')', 'Punctuation'), ('901-1600', 'Number'), ('▲', 'Foreign'), ('정', 'Noun'), ('금선', 'Noun'), ('씨', 'Suffix'), ('별세', 'Noun'), ('ㆍ', 'Foreign'), ('박', 'Noun'), ('남일', 'Noun'), ('(', 'Punctuation'), ('전', 'Noun'), ('대한항공', 'Noun'), ('홍보', 'Noun'), ('실장', 'Noun'), (')', 'Punctuation'), ('남명', 'Noun'), ('(', 'Punctuation'), ('아세아', 'Noun'), ('제지', 'Noun'), ('상무', 'Noun'), (')', 'Punctuation'), ('씨', 'Noun'), ('모친상', 'Noun'), ('=', 'Punctuation'), ('31일', 'Number'), ('오전', 'Noun'), ('8시', 'Number'), (

Komoran
- 장례식장의 경우 징례식과 장으로 분리해버림
- 홍보실장의 경우 홍보와 실장으로 분리해버림
- 대한항공의 경우 하나의 단어로 잡아냄
- 박양순 이름의 경우 박양과 순으로 분리해버림

In [81]:
print(komoran.pos(sent1))
#일반명사, 고유명사, 의존명사 등으로 세부적으로 분리할 수 있으나, 일부 단어의 경우 기존의 원형을 잃어버리는 경우가 발생하였다

[('▲', 'SW'), ('박양', 'NNP'), ('순', 'NNP'), ('씨', 'NNB'), ('별세', 'NNG'), ('ㆍ', 'SP'), ('이해찬', 'NNP'), ('(', 'SS'), ('더불어민주당', 'NNP'), ('의원', 'NNG'), (')', 'SS'), ('씨', 'NNB'), ('모친상', 'NNG'), ('=', 'SW'), ('31', 'SN'), ('일', 'NNB'), ('오전', 'NNG'), ('9', 'SN'), ('시', 'NNB'), ('세종', 'NNP'), ('은하수', 'NNP'), ('장례식', 'NNG'), ('장', 'NNP'), ('발인', 'NNG'), ('2', 'SN'), ('일', 'NNB'), ('오전', 'NNG'), ('8', 'SN'), ('시', 'NNB'), ('(', 'SS'), ('044', 'SN'), (')', 'SS'), ('901', 'SN'), ('-', 'SW'), ('1600', 'SN'), ('▲', 'SW'), ('정', 'NNP'), ('금서', 'NNP'), ('ㄴ', 'ETM'), ('씨', 'NNB'), ('별세', 'NNG'), ('ㆍ', 'SP'), ('박', 'NNP'), ('남일', 'NNP'), ('(', 'SS'), ('전', 'NNP'), ('대한항공', 'NNP'), ('홍보실', 'NNG'), ('장', 'NNP'), (')', 'SS'), ('남명', 'NNP'), ('(', 'SS'), ('아세아', 'NNP'), ('제지', 'NNP'), ('상무', 'NNP'), (')', 'SS'), ('씨', 'NNB'), ('모친상', 'NNG'), ('=', 'SW'), ('31', 'SN'), ('일', 'NNB'), ('오전', 'NNG'), ('8', 'SN'), ('시', 'NNB'), ('46', 'SN'), ('분', 'NNB'), ('일산', 'NNP'), ('병원', 'NNP'), ('발인', 'NNG'), ('2', 'SN

Kkma
- 장례식장의 경우 장례식과 장으로 분리해버림
- 홍보실장의 경우 홍보와 실장으로 분리해버림
- 대한항공의 경우 대한과 항공으로 분리해버림
- 박양순 이름의 경우 한 자씩 분리해버림 

In [82]:
print(kkma.pos(sent1))

[('▲', 'SW'), ('박', 'NNG'), ('양', 'NNG'), ('순', 'NNG'), ('씨', 'NNB'), ('별세', 'NNG'), ('ㆍ', 'UN'), ('이해', 'NNG'), ('찬', 'NNG'), ('(', 'SS'), ('더', 'MAG'), ('붇', 'VV'), ('어', 'ECD'), ('민주당', 'NNG'), ('의원', 'NNG'), (')', 'SS'), ('씨', 'NNB'), ('모친상', 'NNG'), ('=', 'SW'), ('31', 'NR'), ('일', 'NNM'), ('오전', 'NNG'), ('9', 'NR'), ('시', 'NNM'), ('세종', 'NNG'), ('은하수', 'NNG'), ('장례식', 'NNG'), ('장', 'NNG'), ('발인', 'NNG'), ('2', 'NR'), ('일', 'NNM'), ('오전', 'NNG'), ('8', 'NR'), ('시', 'NNM'), ('(', 'SS'), ('044', 'NR'), (')', 'SS'), ('901-1600', 'OL'), ('▲', 'SW'), ('정', 'NNG'), ('금', 'NNG'), ('선', 'NNG'), ('씨', 'NNB'), ('별세', 'NNG'), ('ㆍ', 'UN'), ('박', 'NNG'), ('남', 'NNG'), ('일', 'NNG'), ('(', 'SS'), ('전', 'NNG'), ('대한', 'NNG'), ('항공', 'NNG'), ('홍보', 'NNG'), ('실장', 'NNG'), (')', 'SS'), ('남명', 'NNG'), ('(', 'SS'), ('아세아', 'NNG'), ('제지', 'NNG'), ('상무', 'NNG'), (')', 'SS'), ('씨', 'NNB'), ('모친상', 'NNG'), ('=', 'SW'), ('31', 'NR'), ('일', 'NNM'), ('오전', 'NNG'), ('8', 'NR'), ('시', 'NNM'), ('46', 'NR'), ('분